In [9]:
import numpy as np
import pandas as pd

In [10]:
#Load dataset
dfDataChampionship            =  pd.read_csv("campeonato-brasileiro-full.csv", delimiter=",")

#To String
dfDataChampionship            =  dfDataChampionship.applymap(str);  

#Create new column Placar
dfDataChampionship["Placar"]  =  dfDataChampionship["Clube 1 Gols"].map(str) + "x" + dfDataChampionship["Clube 2 Gols"]

#Show data
dfDataChampionship.head()

,Horário,Dia,Data,Clube 1,Clube 2,Vencedor,Rodada,Arena,Clube 1 Gols,Clube 2 Gols,Clube 1 Estado,Clube 2 Estado,Estado Clube Vencedor,Placar
0,16h00,Sábado,2000-07-29,Fluminense,Bahia,Fluminense,1ª Rodada,Maracanã,2,0,RJ,BA,RJ,2x0
1,16h00,Sábado,2000-07-29,Vasco,Sport,Sport,1ª Rodada,São Januário,0,2,RJ,PE,PE,0x2
2,16h00,Sábado,2000-07-29,Vitória,Palmeiras,Vitória,1ª Rodada,Barradão,4,1,ES,SP,ES,4x1
3,17h00,Domingo,2000-07-30,Botafogo-RJ,Atlético-MG,-,1ª Rodada,Caio Martins,0,0,RJ,MG,Empate,0x0
4,18h30,Domingo,2000-07-30,Juventude,Flamengo,-,1ª Rodada,Alfredo Jaconi,1,1,RS,RJ,Empate,1x1


In [11]:
d = {'Clubes': [], 'PG': [], 'j': [], 'V': [], 'E': [], 'D': [], 'GP': [], 'GC': [], 'SG': []}
dfClassChampionship = pd.DataFrame(data=d);

In [12]:
filter1  =  dfDataChampionship['Data']>='2018-01-01'
filter2  =  dfDataChampionship['Data']<='2018-12-31'
brasilianLeague2018 = dfDataChampionship[filter1 & filter2]
brasilianLeague2018.head()

,Horário,Dia,Data,Clube 1,Clube 2,Vencedor,Rodada,Arena,Clube 1 Gols,Clube 2 Gols,Clube 1 Estado,Clube 2 Estado,Estado Clube Vencedor,Placar
7179,19:00,Sábado,2018-04-14,Vitória,Flamengo,-,1ª Rodada,Barradão,2,2,ES,RJ,Empate,2x2
7180,16:00,Sábado,2018-04-14,Cruzeiro,Grêmio,Grêmio,1ª Rodada,Mineirão,0,1,MG,RS,RS,0x1
7181,21:00,Sábado,2018-04-14,Santos,Ceará,Santos,1ª Rodada,Pacaembu,2,0,SP,CE,SP,2x0
7182,11:00,Domingo,2018-04-15,América-MG,Sport,América-MG,1ª Rodada,Independência,3,0,MG,PE,MG,3x0
7183,19:00,Domingo,2018-04-15,Athlético-PR,Chapecoense,Athlético-PR,1ª Rodada,Arena da Baixada,5,1,PR,SC,PR,5x1


In [13]:
#Cancatenando as colunas Clube 1 e Clube 2
clubs  =  pd.concat([brasilianLeague2018['Clube 1'].str.lower(), brasilianLeague2018['Clube 2'].str.lower()], axis=1, keys=['Clubes'])


In [14]:
#Obtem todos os clubes 
cb  =  pd.Series(clubs['Clubes'].unique(), name="Clubes")
cb  =  cb.to_frame()
cb

,Clubes
0,vitória
1,cruzeiro
2,santos
3,américa-mg
4,athlético-pr
5,corinthians
6,internacional
7,vasco
8,botafogo-rj
9,são paulo


In [15]:
def getNumberWinner(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter = df["Vencedor"].str.lower() == clube
    return (data[filter]['Vencedor'].count()).astype(np.int64)

def getNumberDepartures(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = df["Clube 1"].str.lower() == clube
    filter2 = df["Clube 2"].str.lower() == clube
    return (data[filter1]['Clube 1'].count() + data[filter2]['Clube 2'].count()).astype(np.int64)

def getPoints(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = df["Clube 1"].str.lower() == clube
    filter2 = df["Clube 2"].str.lower() == clube
    filter3 = df["Vencedor"].str.lower() == clube
    filter4 = (df["Clube 1"].str.lower() == clube) | (df["Clube 2"].str.lower() == clube)
    filter5 = df["Vencedor"].str.lower() == '-'
    
    v1 = data[(filter1) & (filter3)]
    v1 = v1['Vencedor'].count()
    v2 = data[(filter2) & (filter3)]
    v2 = v2['Vencedor'].count() 
    v3 = data[(filter4) & (filter5)]
    v3 = v3['Vencedor'].count()
    return ((v2*3)+(v1*3)+v3).astype(np.int64)

def getDraw(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = (df["Clube 1"].str.lower() == clube) | (df["Clube 2"].str.lower() == clube)
    filter2 = df["Vencedor"].str.lower() == '-'
    df = data[(filter1) & (filter2)]
    empates = df['Vencedor'].count()
    return empates.astype(np.int64)

def getDefeats(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = (df["Clube 1"].str.lower() == clube) | (df["Clube 2"].str.lower() == clube)
    filter2 = (df["Vencedor"].str.lower() != clube) & (df["Vencedor"].str.lower() != '-')
    df = data[(filter1) & (filter2)]
    derrotas = df['Vencedor'].count()
    return derrotas.astype(np.int64)

def getGP(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1  =  df["Clube 1"].str.lower() == clube
    filter2  =  df["Clube 2"].str.lower() == clube
    df1      =  data[(filter1)]
    df2      =  data[(filter2)]
    placar1  =  df1['Placar'].str.split('x')
    placar2  =  df2['Placar'].str.split('x')
    
    gp       =  0
    gc       =  0
    for g1, g2 in placar1:
        gp = (gp + pd.to_numeric( g1 ))
        gc = (gc + pd.to_numeric( g2 ))
            
    for g1, g2 in placar2:
        gp = (gp + pd.to_numeric( g2 )) 
        gc = (gc + pd.to_numeric( g1 ))
    return gp, gc

In [16]:

dfTable = cb[["Clubes"]].copy()
for column in ["PG", "J", "V", "E", "D", "GP", "GC", "SG"]:
    dfTable[column] = 0

def ensureUtf(s, encoding='utf8'):
  if type(s) == bytes:
    return s.decode(encoding, 'ignore')
  else:
    return s

for index, row in dfTable.iterrows():
    c1  =  row['Clubes']
    c1  =  ensureUtf( c1 )
    c1  =  c1.strip()
    
    pg       =  getPoints(brasilianLeague2018, c1 )
    j        =  getNumberDepartures(brasilianLeague2018, c1 )
    v        =  getNumberWinner(brasilianLeague2018, c1 )
    e        =  getDraw(brasilianLeague2018, c1 )
    d        =  getDefeats(brasilianLeague2018, c1 )
    gp, gc   =  getGP(brasilianLeague2018, c1 )
    
    if c1 == "sport":
        pg = pg -3
    
    dfTable.at[index, 'PG'] = pg
    dfTable.at[index, 'J']  = j
    dfTable.at[index, 'V']  = v
    dfTable.at[index, 'E']  = e
    dfTable.at[index, 'D']  = d
    dfTable.at[index, 'GP'] = gp
    dfTable.at[index, 'GC'] = gc
    dfTable.at[index, 'SG'] = gp - gc

dfTable['Clubes'] = dfTable['Clubes'].str.capitalize() 
dfTable  =  dfTable.sort_values(by=['PG', 'J', 'V'], ascending=False)
dfTable  =  dfTable.reset_index(drop=True)
dfTable

,Clubes,PG,J,V,E,D,GP,GC,SG
0,Palmeiras,80,38,23,11,4,64,26,38
1,Flamengo,72,38,21,9,8,59,29,30
2,Internacional,69,38,19,12,7,51,29,22
3,Grêmio,66,38,18,12,8,48,27,21
4,São paulo,63,38,16,15,7,46,34,12
5,Atlético-mg,59,38,17,8,13,56,43,13
6,Athlético-pr,57,38,16,9,13,54,37,17
7,Cruzeiro,53,38,14,11,13,34,34,0
8,Botafogo-rj,51,38,13,12,13,38,46,-8
9,Santos,50,38,13,11,14,46,40,6


## Observações

#### Sport - PE: Em julgamento da 2ª CD de 24/04/19, o Sport/PE foi punido com a perda de 3 pontos por infração ao Art. 191,III do CBJD c/c Art. 19§§ 3º e 4º do REC. Em julgamento do Pleno de 08/07/19, foi mantida a perda de 3 pontos.